In [530]:
import requests
import json
import pandas as pd
from datetime import timedelta, date, datetime
import time

addresses = set(
    [
        "0xef6d69922bc2d038cb508086846524f8011c4a74",
        "0xd387a6e4e84a6c86bd90c158c6028a58cc8ac459",
        "0xe1d29d0a39962a9a8d2a297ebe82e166f8b8ec18",
        "0xd5a5c2ef8977ab40d920417018145508251a07c0",
        "0x1b2965dc3b1697dd10b7126ec7393d79cda2cf91",
        "0x78f0269f5b1ca914c8c58993f41181f89e212b27",
        "0x8dbbca57ea56290efa14d835bbfd34faf1d89753",
        "0x53dda0a69992b87e51f0f14b1337281f09a2c8da",
        "0x63a9dbce75413036b2b778e670aabd4493aaf9f3",
        "0x3612b2e93b49f6c797066ca8c38b7f522b32c7cb",
        "0xebf02c6e12c3ee119abba161c40bfeead0a06b15",
        "0xf6bc89cc57e33c4e8053c4d8e63b70bf8c47e1a1",
        "0x530cf036ed4fa58f7301a9c788c9806624cefd19",
        "0x79ecd68a6bcdf40f40bf0dc0c1fcc962aa8c6213",
        "0x6b67623ff56c10d9dcfc2152425f90285fc74ddd",
        "0xe10824fd1ffb9951215025eb26cda42d2d8d9c4c",
        "0x9e199d8a3a39c9892b1c3ae348a382662dcbaa12",
        "0xd5a9c4a92dde274e126f82b215fccb511147cd8e",
        "0xbff79922fcbf93f9c30abb22322b271460c6bebb",
        "0xb53349160e38739b37e4bbfcf950ed26e26fcb41",
    ]
)
events = []
a = None
url = "https://api.opensea.io/api/v1/events"
for address in addresses:
    querystring = {
        "only_opensea": "false",
        "offset": "0",
        "limit": "200",
        "account_address": address,
        "event_type": "successful",
    }
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    events += json.loads(response.text)["asset_events"]
    time.sleep(10)


In [542]:
buy_sell_dict = {"timestamp": [], "asset_name": [], "type": [], "price": []}
for event in events:
    if event["asset"] is None:
        continue
    buy_sell_dict["timestamp"].append(
        datetime.strptime(event["transaction"]["timestamp"], "%Y-%m-%dT%H:%M:%S")
    )
    buy_sell_dict["asset_name"].append(event["asset"]["asset_contract"]["name"])
    buy_sell_dict["type"].append(
        "sell" if event["winner_account"]["address"] not in addresses else "buy"
    )
    buy_sell_dict["price"].append(float(event["total_price"]) / 10e17)

buy_sell_df = pd.DataFrame.from_dict(buy_sell_dict)

buy_sell_df


,timestamp,asset_name,type,price
0,2021-09-06 00:43:19,Autoglyphs,sell,375.0000
1,2021-08-30 18:38:59,Meebits,sell,9.0000
2,2021-08-25 18:31:34,Wrapped Cryptopunks,sell,80.0000
3,2021-08-06 18:16:51,Meebits,sell,15.0000
4,2021-08-05 16:22:42,CryptoPunks,sell,99.0000
...,...,...,...,...
3635,2021-07-07 15:19:13,Art Blocks,buy,0.1150
3636,2021-07-06 18:00:06,CryptoPunks,sell,19.0000
3637,2021-07-06 17:18:42,Art Blocks,buy,0.0899
3638,2021-07-05 20:11:29,OpenSea Collection,buy,0.1000


In [554]:
top_buy_by_count = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "buy")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)
top_sell_by_count = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "sell")
    ][["asset_name", "type"]]
    .groupby(["asset_name", "type"])
    .size()
    .reset_index(name="counts")
    .sort_values("counts", ascending=False)
)


In [566]:
top_buy_by_count

,asset_name,type,counts
1,Cranes,buy,13
2,Emblem Vault V2,buy,11
0,Art Blocks,buy,3
4,OpenSea Collection,buy,2
3,EtherpoemsSpokenWord,buy,1
5,TinyBoxes,buy,1


In [578]:
top_sell_by_count

,asset_name,type,counts
0,APE DAO REMIX!,sell,23
6,CryptoKitties,sell,14
4,Axolittles,sell,11
16,PLUTO,sell,9
10,GalaxyEggs,sell,8
14,MonkeyBet,sell,6
13,Groupies,sell,5
17,Sappy Seals,sell,5
18,Sneaky Vampire Syndicate,sell,5
11,GalaxyFightClub,sell,4


In [590]:
top_buy_by_price = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "buy")
    ][["asset_name", "type", "price"]]
    .groupby(["asset_name", "type"])
    .sum()
    .sort_values("price", ascending=False)
)
top_sell_by_price = (
    buy_sell_df[
        (buy_sell_df["timestamp"] >= datetime.now() - timedelta(days=1))
        & (buy_sell_df["type"] == "sell")
    ][["asset_name", "type", "price"]]
    .groupby(["asset_name", "type"])
    .sum()
    .sort_values("price", ascending=False)
)

In [602]:
top_buy_by_price

,,price
asset_name,type,
Emblem Vault V2,buy,236.784800
Art Blocks,buy,13.644096
Cranes,buy,0.605000
EtherpoemsSpokenWord,buy,0.198000
OpenSea Collection,buy,0.175000
TinyBoxes,buy,0.100000


In [614]:
top_sell_by_price

,,price
asset_name,type,
Art Blocks,sell,2.770000e+01
APE DAO REMIX!,sell,1.774300e+01
GalaxyEggs,sell,6.025000e+00
Frontier,sell,5.610000e+00
Sneaky Vampire Syndicate,sell,5.550000e+00
Axolittles,sell,3.837000e+00
MonkeyBet,sell,3.342000e+00
Creature World,sell,1.930000e+00
GalaxyFightClub,sell,1.778000e+00


In [929]:
hist_dict = {
    "timestamp": [],
    "asset_id": [],
    "asset_name": [],
    "token_id": [],
    "seller": [],
    "buyer": [],
    "type": [],
    "price": [],
    "permalink": [],
}
for event in events:
    if event["asset"] is None:
        continue
    hist_dict["timestamp"].append(
        datetime.strptime(
            event["transaction"]["timestamp"]
            if event["transaction"]
            else event["created_date"],
            "%Y-%m-%dT%H:%M:%S",
        )
    )
    hist_dict["asset_id"].append(event["asset"]["id"])
    hist_dict["asset_name"].append(event["asset"]["asset_contract"]["name"])
    hist_dict["token_id"].append(event["asset"]["token_id"])
    hist_dict["seller"].append(
        event["seller"]["user"]["username"]
        if event["seller"] and event["seller"]["user"]
        else None
    )
    hist_dict["buyer"].append(
        event["winner_account"]["user"]["username"]
        if event["winner_account"]["user"] is not None
        else None
    )
    hist_dict["type"].append(
        "sell" if event["winner_account"]["address"] not in addresses else "buy"
    )
    hist_dict["price"].append(float(event["total_price"]) / 10e17)
    hist_dict["permalink"].append(
        event["asset"]["permalink"] if "permalink" in event["asset"] else None
    )

hist_df = pd.DataFrame.from_dict(hist_dict)


def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)


In [940]:
# top_buy
hist_df[
    (hist_df["timestamp"] >= datetime.now() - timedelta(days=1))
    & (hist_df["type"] == "buy")
].sort_values("price", ascending=False).style.format({"permalink": make_clickable})


,timestamp,asset_id,asset_name,token_id,seller,buyer,type,price,permalink
2134,2021-09-18 01:05:53,51270229,Emblem Vault V2,33525106800726861,Pepito888,KeyboardMonkeyVault,buy,144.000000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/33525106800726861
2135,2021-09-18 00:52:42,53055645,Emblem Vault V2,6129198475786001,NichDidmore,KeyboardMonkeyVault,buy,35.000000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/6129198475786001
2137,2021-09-18 00:49:26,53456928,Emblem Vault V2,27296779429703821,Memerise,KeyboardMonkeyVault,buy,27.000000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/27296779429703821
2136,2021-09-18 00:51:24,54184753,Emblem Vault V2,15920971443855241,-Rare_Designer-,KeyboardMonkeyVault,buy,15.000000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/15920971443855241
2131,2021-09-18 02:51:59,41000552,Emblem Vault V2,9051501,topbuyerpleb,KeyboardMonkeyVault,buy,14.750000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/9051501
2530,2021-09-17 13:50:53,53141426,Art Blocks,162000545,bluekoolaiduno,VonMises,buy,7.000000,https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/162000545
2529,2021-09-17 13:53:27,53142462,Art Blocks,162001008,GeorgeMcFly,VonMises,buy,5.734096,https://opensea.io/assets/0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270/162001008
1930,2021-09-17 01:43:16,41336785,Emblem Vault V2,6173951,AugustaFarms,punkopoly,buy,0.210000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/6173951
265,2021-09-17 23:17:32,30869489,EtherpoemsSpokenWord,86,SmARTcollector,dan_,buy,0.198000,https://opensea.io/assets/0x8fdde660c3ccab82756acc5233687a4ceb4b8f30/86
1931,2021-09-17 01:36:41,49181124,Emblem Vault V2,49369214810419821,rasher,punkopoly,buy,0.188000,https://opensea.io/assets/0x82c7a8f707110f5fbb16184a5933e9f78a34c6ab/49369214810419821


In [951]:
# top sell
hist_df[
    (hist_df["timestamp"] >= datetime.now() - timedelta(days=1))
    & (hist_df["type"] == "sell")
].sort_values("price", ascending=False).style.format({"permalink": make_clickable})


,timestamp,asset_id,asset_name,token_id,seller,buyer,type,price,permalink
782,2021-09-17 07:46:09,17185658,Art Blocks,3143,None,None,sell,17.000000,https://opensea.io/assets/0x059edd72cd353df5106d2b9cc5ab83a52287ac3a/3143
781,2021-09-17 17:13:29,17173148,Art Blocks,2446,None,cryptostorm,sell,10.700000,https://opensea.io/assets/0x059edd72cd353df5106d2b9cc5ab83a52287ac3a/2446
2140,2021-09-17 21:05:41,44646035,Frontier,1,KeyboardMonkeyVault,Insayne,sell,2.680000,https://opensea.io/assets/0x236672ed575e1e479b8e101aeeb920f32361f6f9/1
2743,2021-09-18 04:01:17,53996147,APE DAO REMIX!,2151,0x541nt,xodivv,sell,2.450000,https://opensea.io/assets/0xfb61bd914d4cd5509ecbd4b16a0f96349e52db3d/2151
2762,2021-09-17 20:25:05,42169146,Creature World,7906,0x541nt,jgrunt,sell,1.930000,https://opensea.io/assets/0xc92ceddfb8dd984a89fb494c376f9a48b999aafc/7906
2752,2021-09-17 23:36:52,53996139,APE DAO REMIX!,2156,0x541nt,jpegmillionaire,sell,1.850000,https://opensea.io/assets/0xfb61bd914d4cd5509ecbd4b16a0f96349e52db3d/2156
2757,2021-09-17 22:24:17,54002276,APE DAO REMIX!,3389,0x541nt,ThunderRunner24,sell,1.839000,https://opensea.io/assets/0xfb61bd914d4cd5509ecbd4b16a0f96349e52db3d/3389
2754,2021-09-17 23:19:35,54003862,APE DAO REMIX!,3834,0x541nt,0xSkG,sell,1.830000,https://opensea.io/assets/0xfb61bd914d4cd5509ecbd4b16a0f96349e52db3d/3834
2730,2021-09-18 06:19:37,53995150,APE DAO REMIX!,1954,0x541nt,None,sell,1.790000,https://opensea.io/assets/0xfb61bd914d4cd5509ecbd4b16a0f96349e52db3d/1954
2139,2021-09-17 21:29:54,44646035,Frontier,1,KeyboardMonkeyVault,BilboMcBaggins,sell,1.590000,https://opensea.io/assets/0x236672ed575e1e479b8e101aeeb920f32361f6f9/1
